# 6. 작사가 인공지능 만들기

목차
시퀀스? 스퀀스!
I 다음 am을 쓰면 반 이상은 맞더라
실습
1) 데이터 다듬기
2) 인공지능 학습시키기
3) 잘 만들어졌는지 평가하기
프로젝트 : 멋진 작사가 만들기

# 6-2. 시퀀스? 스퀀스!

## 시퀀스
- 시퀀스는 데이터에 순서(번호)를 붙여 나열한 것
  
- 특징:
    - 데이터를 순서대로 하나씩 나열하여 나타낸 구조다
    - 특정 위치(~번째) 데이터를 가리킬 수 있다.
    
시퀀스가 담은 데이터 순서는 정렬이 아닌 나열이다. 

# 6-3. I 다음 am을 쓰면 반 이상은 맞더라

나는 밥을 [ ] = 먹는다 ; 밥은 통계적으로 먹힘
알바생이 커피를 [ ] = 만든다 ; 알바생은 통계적으로 커피를 만듦

인공지능이 글을 읽는 방식도 같다. 문법적인 원리를 통해서가 아닌 수많은 글을 읽게 함으로써 나는, 밥을, 뒤에는 먹는다가, 알바생이, 커피를, 뒤에는 만든다라느 사실을 알게 하는 것.
  
그런 이유에서 많은 데이터가 곧 좋은 결과를 만들어냄
  
이 방식을 제일 잘 처리하는 인공지능 중 하나가 순환신경망(RNN)임
![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-12-RNN2.max-800x600.png)

나는 밥을 [ ] = 먹는다가 들어가야 하는 것을 배웠지만  
첫 시작인 '나는'은 어떻게 만드나?
  
\<strat> 라는 특수한 토큰을 맨 앞에 추가해주므로써 해결 가능. \<start>는 "이제 문장을 생성해봐"라고 사인을 주는 것  
\<strat>를 입력 받은 순환신경망(RNN)은 다음 단어로 '나는'을 생성하고, **생성한 단어를 다음 입력으로 사용**  
이런 순환적인 특성을 살려 이름이 순환신경망(RNN)
    
그렇게 순차적으로 '밥을 먹었다'까지 생성하고 나면 인공지능은 "다 만들었다"는 신호로 \<end> 토큰을 생성.  
즉, 우리는 \<strat>가 문장의 시작에 더해진 데이터(**문제지**)와 \<end>가 문장의 끝에 더해진 출력(**답안지**)가 필요함  
이는 문장 데이터만 있으면 만들어낼 수 있다는 것

In [39]:
sentence = "나는 밥을 먹었다"

source_sentence = "<start>" + sentence
target_sentence = sentence + '<end>'

print("Source 문장 : ", source_sentence)
print("Target 문장 : ", target_sentence)

Source 문장 :  <start>나는 밥을 먹었다
Target 문장 :  나는 밥을 먹었다<end>


# 언어 모델 (Language Model)

나는, 밥을, 먹었다를 순차적으로 생성할 때 '밥을' 다음이 '먹었다'인 것은 쉽게 알 수 있다.  
하지만 '나는' 다음이 '밥을'인 것은 조금 억지스러움  
실제 동작 방식에서도 '나는' 다음이 '밥을'이 나오는 것은 순전히 운

## 사건 B가 일어나는 경우에 사건 A가 일어날 확률은 P(A \vert B)P(A∣B)로 표기
  
확률적 표현:  
'나는 밥을' 다음에 '먹었다'가 나올 확률을 p(먹었다|나는 밥을)  
그렇다면 이 확률은 '나는' 뒤에 '밥이'가 나올 확률인 p('밥이'|'나는')보다는 높게 나올 것  
아마 p(먹었다|나는, 밥을, 맛있게)의 확률값은 더 높을 것  
어떤 문구 뒤에 다음 단어가 나올 확률이 높다는 것은 다음 단어가 나오는 것이 보다 자연스럽다는 뜻  
나는 뒤에 밥을이 나오는 것이 자연스럽지 않다는 것이 아니라 나는 뒤에는 올 수 있는 자연스러운 단어의 경우의 수가 너무 많다보니 불확실성이 높을 뿐

n-1개의 단어 시퀀스 w_1,...,w_n-1가 주어졌을 때 n번째 단어 w_n으로 무엇이 올지를 예측하는 확률 모델을 언어모델(Language Model)이라고 부름  
P(w_n|w_1,...,w_n-1;θ) : w_n-1 다음에 w_n이 나올 확률  
n-1번재까지의 단어 시퀀스가 x_train이 되고 n번째 단어가 y_train
w_1, ... , w_n-1가 주어졌을 때 n번재 단어 w_n으로 무엇이 올지 예측하는 구조를 가지고 있음을 알아챌 수 있음

# 6-4. 실습 (1) 데이터 다듬기

In [40]:
import re       # 정규표현식을 위한 Regax 지원 모듈(문장 데이터 정돈에 필요)
import numpy as np       # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf   # 텐서플로우
import os

# 파일 읽기모드로 열기
file_path = os.getenv("HOME") + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, 'r') as f:
    raw_corpus = f.read().splitlines() # 텍스트를 라인 단위로 끊어서 list 형태로 불러옴
    
print(raw_corpus[:9]) # 앞에서부터 10라인만 화면에 출력

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


### 우린 문장(대사)만을 원하므로 화자 이름이나 공백뿐인 정보는 필요가 없음
#### 1차 필터링
![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-12-3.data.max-800x600.png)
  
### 원치 않는 문장  
#### 화자가 표기된 문장(0, 3, 6), 공백인(2, 5, 9)  
##### 화자가 표기된 문장은 문장의 끝이 ':'로 끝남. 일반적으로 대사가 ':'로 끝날일이 없으니 ':'를 제외시켜도 괜찮을듯
##### 공백인 문장은 길이를 검사하여 길이가 0이라면 제외

In [42]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue # 길이가 0인 문장은 건너뛰고
    if sentence[-1] == ":": continue # 문장의 끝이 :이면 건너뛰고
    
    if idx >9: break # 일단 문장 10개에서 끝내
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


### 텍스트 생성 모델에도 단어사전을 만들게 됨
### 그렇다면 문장을 일정한 기준으로 쪼개야함
### 일정한 기준으로 쪼개는 과정을 토큰화(Tokenize) 라고 함
  
### 문제케이스  
1. Hi, my name is John. ('hi', 'my', ... 'john.')으로 분리됨 - 문장부호  
2. First, epn the first chapter. (First와 first 다른 단어로 인식) - 대소문자  
3. He is a ten-year-old boy. (ten-year-old를 한 단어로 인식) - 특수무자  
  
'1.'을 막기 위해 문장 부호 양쪽에 공백을 추가  
'2.'를 막기 위해 모든 문자들을 소문자로 변환  
'3.'을 막기 위해 특수문자들은 모두 제거

## 이런 전처리 정규표현식(Regex)을 이용한 필터링이 유용하게 사용됨

In [48]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter(구분문자)로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


# 데이터 전처리 과정

## 우리가 구축해야할 데이터 셋은 어떤 모양일까?
자연어 처리 분야에서  
소스문장(Source sentence) : 모델의 입력이 되는 문장 : x_train  
타겟문장(Target sentence) : 정답 역할을 하게 될 모델의 출력 문장 : y_train    
    
위에서 만든 정제 함수를 통해 만든 데이터셋 토큰화를 진행한 후  
끝 단어 '<end>'를 없애면 소스문장  
첫 단어 '<start>'를 없애면 타겟문장이 됨

In [49]:
# 정제함수 활용하여 정제 데이터 구축

corpus = []

for sentence in raw_corpus:            # row_corpus를 Sentece에 반복해서 넣어라
    if len(sentence) == 0: continue    # sentence길이가 0이면 건너뛰고
    if sentence[-1] == ":": continue   # sentece 끝이 : 이면 건너뛰고
        
    corpus.append(preprocess_sentence(sentence))   # courpus에 추가해라 만든 정제함수(preprocess_sentece)에 sentence를 넣어서
    
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

### 텍서플로우는 자연어 처리를 위한 여러가지 모듈을 제공
#### tf.keras.preprocessing.text.Tokenizer 패키지는 
#### 1. 정제된 데이터를 토큰화하고
#### 2. 단어사전(vocabulary, dictionary라고 부름)을 만들어주며
#### 3. 데이터를 숫자로 변환시켜줌

#### 이 과정을 벡터화(vetorize)라 하며, 숫자로 변환된 데이터를 텐서(tensor)라고 함
#### 우리가 사용하는 텐서플로우로 만든 모델의 입출력 데이터는 실제로 모두 이런 텐서 변환되어 처리 되는 것

In [29]:
import pandas as pd
from pandas import DataFrame as df

df1 = df(data={"Rank": ["0", "1", "2", "3", "n"], "Type": ["scalar", "vector", "matrix", "3-tensor", "n-tensor"], 
               "Example": [[1], [1, 1], [[1, 1], [1,1]], [[[1,1],[1,1]],[[1,1],[1,1]],[[1,2],[2,1]]], []]},
        columns = ['Rank', "Type", "Example"])
df1

,Rank,Type,Example
0,0,scalar,[1]
1,1,vector,"[1, 1]"
2,2,matrix,"[[1, 1], [1, 1]]"
3,3,3-tensor,"[[[1, 1], [1, 1]], [[1, 1], [1, 1]], [[1, 2], ..."
4,n,n-tensor,[]


스칼라는 일반적오르 존재하는 그냥 값 1개.  
벡터는 스칼라가 여러개 모인 것이며 차원이 높아질 수록 아래 차원의 것을 모아 놓은 배열이라고 할 수 있음
![img](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FqS1NF%2FbtqubStze09%2F5sbnC8DVd3aQsUULgjv6Kk%2Fimg.png)

In [50]:
def tokenize(corpus):  
    # 텐서플로우에서 제공하는 Tokenizer패키지 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 7000, # 전체 단어 개수
        filters=' ', # 별도로 전처리 로직을 추가할 수 있지만 이번에는 사용 안 함
        oov_token="<unk>" # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus) # 우리가 구축한 corpus로부터 Tokenize가 사전을 자동 구축
    
    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋 구축
    tensor = tokenizer.texts_to_sequences(corpus)  # tokenizer는 구축한 사전으로부터 corpus를 해석새 Tensor로 변환
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding 메소드 제공
    # maxlen의 디폴트 값은 None. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰짐
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fae9199aad0>


In [51]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력
print(tensor[:3,:10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [52]:
# 텐서데이터는 모두 정수로 이루어짐
# 이 숫자는 tokenizer에 구축된 단어 사전의 인덱스
# 단어 사전 어떻게 구축되었는지 확인

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


# 2번째 인덱스가 <start>여서 모든 행이 2로 시작하는 것을 알 수 있음

### 위에 텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 저해진 입력 시퀀스 길이보다 문장이 짧을경우 0으로 패딩(padding)을 채워 넣은 것
### 사전에는 없지만 0은 바로 패딩 문자<pad>가 될 것

In [53]:
src_input = tensor[:, :-1] # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
tgt_input = tensor[:, 1:] # tensor에서 <satrt>를 잘라내서 타겟문장 생성

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


### corpus내의 첫번째 문장에 대해 생성된 소스와 타겟 문장 확인
### 예상대로 소스는 2\<start>에서 시작해서 3\<end>로 끝난 후 0\<pad>로 채워져 있음
### 하지만 타겟은 2로 시작하지않고 소스를 오니쪽으로 한칸 시프트한 형태를 보임(143 부터 시작)

In [ ]:
### 데이터셋 객체 생성
### 그동안 model.fit(x_train, y_train, ...) 형태로 Numpy Array데이터셋을 생성하여 model에 제공하는 형태의 학습을 진행했음
### 그러나 텐서플로우를 활용할 경우 텐서로 생성된 데이터를 이용해 tf.data.Dataset 객체를 많이 사용함
### rf.data.Dataset객체는 텐서플로우에서 활용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의기능을 제공하므로 사용법 알아둘 것

In [ ]:
# 우리는 이미 데이터셋을 텐서 형테로 생성해 두었으므로 tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset 객체 생성

In [54]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1 # tokenizer가 구축된 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>!

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

In [ ]:
# 과정 기억
- 정규표현식을 이용한 corpus 생성
- tf.kera.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환
- tf.data.Dataset.from_tensor_slices()를 이용해 corpus텐서를 tf.data.Dataset객체로 변환

In [ ]:
# dataset을 얻음으로써 데이터 다듬기 과정 끝
# tf.data.Dataset에서 제공하는 shuffle(), batch() 등 다양한 데이터셋 관련 기능을 손쉽게 이용 가능
# 이 모든 과정을 텐서플로우의 데이터 전처리 과정

---

# 6-5. 실습 (2) 인공지능 학습시키기

# 우리가 만들 모델 구조도
![Img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-12-4.max-800x600.png)
#### 우리가 만들 모델은 tf.keras.Model을 Subclassing하는 방식으로 만들 거임
#### 위 그림에서 처럼 우리가 만들 모델에는 1개의 Embedding레이어, 2개의 LSTM레이어, 1개의 DENSE레이어로 구성되어 있음

In [58]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 #embedding_size는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
hidden_size = 1024 # hidden_size는 모델에 얼마나 많은 일꾼을 둘 것인가 정도로 이해하면 됨
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

print('ok')

ok


#### 입력 텐서에는 단어 사전의 인덱스가 들어있음
#### Embedding 레이어는 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔줌
#### 이 워드 벡터는 공간에서 단어의 추상적 표현으로 사용됨
  
#### 위 코드에서 embedding_size는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
#### 만약 그 크기가 2라면
- 차갑다:[0.0, 1.0]
- 뜨겁다:[1.0, 0.0]
- 미지근하다:[0.5, 0.5]
#### 정도로 구분이 가능함
#### embedding_size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만, 그만큼 충분한 데이터가 주어지지 않으면 혼란을 야기
#### 이번 실습에선 256이 적당해보임
  
#### LSTM레이어의 hidden state의 차원수인 hidden_size도 같은 맥락
#### hidden_size는 모델에 얼마나 많은 일꾼을 둘 것인가 정도로 이해하면 됨
#### 마찬가지로 충분한 데이터가 주어지면 올바른 경정을 내리지만 그렇지 않으면 혼란을 야기함

#### 우리의 model은 아직 제대로 build되지 않음
#### model.compile()을 호출한 적도 없고
#### 입력텐서가 무엇인지 제대로 지정도 안해줌
#### 그런경우 아래와 같이 model에 데이터를 아주 조금 태워보는 것도 방법
#### model의 input shape까 결정되면서 model.build까 자동 호출 됨

In [59]:
# model에 데이터 태우기

for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 3.02529457e-04,  5.12329570e-05, -7.60959665e-05, ...,
          8.18049216e-07,  2.62600573e-04,  5.22834671e-05],
        [ 4.41622105e-04, -1.85308512e-04, -1.35317663e-04, ...,
         -5.73762263e-05, -9.03309774e-05,  2.18827263e-04],
        [ 2.61754409e-04, -3.57642042e-04, -3.60564765e-04, ...,
         -1.80342176e-04, -3.41744279e-04,  4.58278955e-04],
        ...,
        [-9.27285175e-04,  1.08358763e-04,  9.16174205e-04, ...,
          3.14459298e-03, -1.67383137e-03, -1.72942516e-03],
        [-8.86218855e-04, -6.15459567e-06,  1.01439608e-03, ...,
          3.54310567e-03, -1.88168488e-03, -2.13172939e-03],
        [-7.93857093e-04, -1.19667275e-04,  1.09925866e-03, ...,
          3.89152835e-03, -2.03005224e-03, -2.47139996e-03]],

       [[ 3.02529457e-04,  5.12329570e-05, -7.60959665e-05, ...,
          8.18049216e-07,  2.62600573e-04,  5.22834671e-05],
        [ 5.99050312e-04,  2.74614518e-04,  3.

#### "<tf.Tensor: shape=(256, 20, 7001)"임을 알 수 있음.
#### 7001은 Dense 레이어의 출력 차원수. 7001개 단어 중 어느 단어의 확률이 가장 높을지를 모델링해야하기 때문
#### 256은 이전 스텝에서 지정한 배치 사이즈
#### dataset.take(1)를 통해 1개의 배치, 즉 256개의 문장을 가져온 것
#### 20은 tf.keras.layers.LSTM(hidden_size, return_sequences=True)로 호출한 LSTM 레이어에서 return_sequences=True라고 지정한 부분에 있음
#### return_sequences=True 뜻 : 즉, LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력한다는 의미
#### return_sequences=False이면 LSTM 레이어는 1개의 벡터만 출력

#### 문제는 우리의 모델은 입력 데이터의 시퀀스 길이가 얼마인지 모름
#### 그럼 20은 언제 알게 된 것일까? -> 데이터를 입력받으면서 알게됨. 우리 데이터셋의 max_len이 20으로 맞춰져 있던 것

In [60]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# model.summary()호출 가능
# 그러나 output shape가 없음
# 왜냐하면 우리의 모델은 입력 시퀀스의 길이를 모르기때문에 output shape를 특정할 수 없음
# 하지만 모델의 파라미터 사이즈는 측정 가능. 약 2천2백만

In [61]:
# 모델 학습 시작
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 12s 132ms/step - loss: 3.4619
Epoch 2/30
93/93 [==============================] - 12s 132ms/step - loss: 2.7997
Epoch 3/30
93/93 [==============================] - 13s 134ms/step - loss: 2.7038
Epoch 4/30
93/93 [==============================] - 12s 133ms/step - loss: 2.6135
Epoch 5/30
93/93 [==============================] - 12s 134ms/step - loss: 2.5469
Epoch 6/30
93/93 [==============================] - 13s 136ms/step - loss: 2.4970
Epoch 7/30
93/93 [==============================] - 13s 136ms/step - loss: 2.4437
Epoch 8/30
93/93 [==============================] - 13s 135ms/step - loss: 2.3972
Epoch 9/30
93/93 [==============================] - 13s 136ms/step - loss: 2.3550
Epoch 10/30
93/93 [==============================] - 13s 136ms/step - loss: 2.3146
Epoch 11/30
93/93 [==============================] - 13s 137ms/step - loss: 2.2741
Epoch 12/30
93/93 [==============================] - 13s 136ms/step - loss: 2.2357
Epoch 13/30
9

In [ ]:
# loss는 모델이 오답을 만들고 있는 정도로 해석하면 됨
# 그렇다고 loss가 1일때 99%를 맞추고 잇는 것은 아님
# 오답율이 감소하고 있으니 학습이 잘 진행되고 있다 정도로만 해석

# 6-6. 실습 (3) 잘 만들어졌는지 평가하기 

In [73]:
# generate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행함

def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해 입력받은 init_sentence도 일단 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성 해야함
    while True:
        predict = model(test_tensor) # 입력받은 문장의 텐서를 입력
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] # 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됨
        
        # 모델이 새롭게 예측한 단어를 입력 문장 뒤에 붙여줌
        test_tensor = tf.concat([test_tensor, 
                                 tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        # 모델이 <end>를 예측했거나, max_len에 도달하지 않았으면 while루프를 또 돌면서 다음 단어를 예측
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
            
    generated = ""
              
    # 생성된 tensor안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated # 이것이 최종적으로 모델이 생성한 자연어 문장
print('ok')

ok


In [ ]:
# 학습단계에서는 while문이 필요 없었음. 
# 소스문장과 타겟문장이있고 우리는 소스문장을 모델에 입력해서 나온 결과를 타겟문장과 비교하기만 하면 됐음
# 그러나 텍스트를 실제로 생성해야하는 시점에서 우리는 2가지가 없음 1. 타겟문장, 2. 소스문장
# 우리는 텍스트 생성 태스크를 위해 테스트 데이터셋을 따로 생성한 적이 없음

# generat_text()함수에서 init_sentence를 인자로 받고 있음
# 이렇게 받은 인자를 일단 텐서로 만들고 디폴트로는 <start> 단어 하나만 받음

# 1. while의 첫번째 루프에서 test_tensor에 <start> 들어갔다고 가정, 우리의 모델이 출력으로 7001개의 단어 중 A를 골랐다고 하면
# 2. while의 두번째 루프에서 test_tensor에는 <start> A가 들어감. 그래서 우리의 모델이 그 다음 B를 골랐다고 하면
# 3. while의 세번째 루프에서 test_tensor에는 <start> A B가 들어감 그래서 .....

In [74]:
# 위 문장 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> home")

'<start> home to the king s ship , sir , that you have been so <end> '

# 6-7. 프로젝트: 멋진 작사가 만들기

## 데이터 준비

In [ ]:
# 데이터 준비
# wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
# unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기

## 데이터 읽어오기

In [240]:
# 데이터 읽어오기
# glob 모듈을 사용하면 파일 읽어오는 작업 하기 용이함
# glob을 활용하여 모든 txt파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장

import glob
import re       # 정규표현식을 위한 Regax 지원 모듈(문장 데이터 정돈에 필요)
import numpy as np       # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf   # 텐서플로우
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러 개의 txt 파일을 모두 읽어서 raw_corpus에 담김
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기: ", len(raw_corpus))
print("Examples:\n", raw_corpus[100:1000], end="\t")

데이터 크기:  187088
Examples:
 ["you're driving me insane", 'Crazy, baby', 'Oh, girl', "Crazy 'bout your love", '', 'Soft and wet (U know)', 'You are soft and wet (Oh, sugar)', 'Your love is soft and wet', 'Soft and wet', '', '', 'You have got a strange way about you', "Kinda' crazy but I love you just the same", 'Because you, oh you make me wanna do, oh everything', '', "I'm just a crazy fool, lost in the world of love", "I get from crazy you, oh, I'm so strung out", "Thinking 'bout the crazy things you do, crazy you", '', '', "You can live your own life and I'll live mine", 'I will never try to keep you down', 'And even if I only see you some of the time', "I'm just happy when you come around", "And even if the sun don't shine", '', "I'm warm enough when", "You're in these arms of mine", "Just as long as we're together", "Every thing's alright, every thing's alright", '', 'There is nothing that will overcome the love we share', 'Nothing that will break us apart', 'Girl, I gotta always ha

## 데이터 정제

In [ ]:
## 데이터 정제
## preprocess_sentence() 함수 활용
## 긴 문장은 과도한 padding을 갖게 하므로 제거.
## 그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외

In [244]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue # 공백 건너뜀
    if sentence[0] == "'": continue # 문장 시작이 ' 건너뜀
    if sentence[-1] == "'": continue # 문장 끝이 ' 건너뜀
    if sentence[0] == '"': continue # 문장 시작이 " 건너뜀
    if sentence[-1] == '"': continue # 문장 끝이 " 건너뜀
    if sentence[0] == '(': continue # 문장 끝이 ( 건너뜀
    if sentence[-1] == ')': continue # 문장 끝이 ) 건너뜀
    if sentence[0] == '[': continue # 문장 끝이 [ 건너뜀
    if sentence[-1] == ']': continue # 문장 끝이 ] 건너뜀        
    
    
#     if idx >= 15: continue # 15개 넘어가는 문장 건너뜀 : <=15는 15이하여서 15개 이하 건너 뛰는 거 아닌가? 
    if idx >10: break # 일단 문장 10개에서 끝내
        
    print(sentence)

SyntaxError: unexpected character after line continuation character (<ipython-input-244-7d00a9203009>, line 11)

In [232]:
len(sentence)

31

In [348]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고 양쪽 공백 삭제
    
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter(구분문자)로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    
    sentence = sentence.strip()
     
    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    sentence = list(sentence.split())
    len(sentence) < 15
    return sentence


print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

['<start>', 'this', 'is', 'sample', 'sentence', '.', '<end>']


In [339]:
result = list(preprocess_sentence(sentence.split()))


AttributeError: 'list' object has no attribute 'lower'

In [317]:
# split()
# result = preprocess_sentense
# ()
# len(result.split())

In [341]:
# 정제함수 활용하여 정제 데이터 구축
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue # 공백 건너뜀
    if len(sentence) == '.': continue # . 건너뜀
    if sentence[0] == "'": continue # 문장 시작이 ' 건너뜀
    if sentence[-1] == "'": continue # 문장 끝이 ' 건너뜀
    if sentence[0] == '"': continue # 문장 시작이 " 건너뜀
    if sentence[-1] == '"': continue # 문장 끝이 " 건너뜀
    if sentence[0] == '(': continue # 문장 끝이 ( 건너뜀
    if sentence[-1] == ')': continue # 문장 끝이 ) 건너뜀
    if sentence[0] == '[': continue # 문장 끝이 [ 건너뜀
    if sentence[-1] == ']': continue # 문장 끝이 ] 건너뜀  
    if sentence < 30: continue # 15개 넘어가는 문장 건너뜀 # 이건 글자가 30자 미만인거고 단어가 30개 미만이려면?
        
    corpus.append(preprocess_sentence(sentence)) # corpus에 추가해라 만든 정제함수(preprocess_sentece)에 sentence를 넣어서

In [342]:
print(type(corpus[2]))
print(len(corpus[2]))

<class 'list'>
8


# 평가 데이터셋 분리

In [ ]:
## 평가 데이터셋 분리
## tokenize() 함수로 데이터를 tensor로 변환한 후 
## sklearn 모듈의 train_test_split()함수를 사용해 훈련데이터와 평가데이터 분리
## 단어장의 크기는 12,000이상으로 설정, 총 데이터의 20%를 평가 데이터셋으로 사용
## enc_train, enc_val, dec_train, dec_val = <코드 작성>

In [343]:
def tokenize(corpus):  
    # 텐서플로우에서 제공하는 Tokenizer패키지 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000, # 전체 단어 개수
        filters=' ', # 별도로 전처리 로직을 추가할 수 있지만 이번에는 사용 안 함
        oov_token="<unk>" # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus) # 우리가 구축한 corpus로부터 Tokenize가 사전을 자동 구축

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋 구축
    tensor = tokenizer.texts_to_sequences(corpus)  # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding 메소드 제공
    # maxlen의 디폴트 값은 None. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰짐
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post') # 토큰화 15개 미만?
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

print('ok')


[[  2  25  19 ...   0   0   0]
 [  2  29  36 ...   0   0   0]
 [  2  13 103 ...   0   0   0]
 ...
 [  2   5  22 ...   0   0   0]
 [  2  25   6 ...   0   0   0]
 [  2  82   4 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f29fbaf9710>
ok


In [345]:
len(tokenizer.index_word)

26956

In [346]:
print(type(tensor.shape))
print(type(tokenizer))

<class 'tuple'>
<class 'keras_preprocessing.text.Tokenizer'>


In [347]:
# 생성된 텐서 데이터를 출력
print(tensor[1])

[    2    29    36     4 10949     8  3804   280     3     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [323]:
# skelarn의 train_test_split 함수를 통해 train_data와 test_data분리  
# test_size 인자를 조절해주면 설정해준 값 만큼 test dataset 비율 조정 가능
# from sklearn.datasets import corpus
from sklearn.model_selection import train_test_split 


enc_train, enc_val, dec_train, dec_val = train_test_split(tensor, tensor, test_size=0.2, shuffle=True)


In [324]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print(enc_val.shape)
print(dec_val.shape)

Source Train: (129334, 347)
Target Train: (129334, 347)
(32334, 347)
(32334, 347)


In [ ]:
## 위 과정까지 올바르게 진행했을 경우 아래의 실행결과 확인 가능
# print("Source Train : ", enc_train.shape)
# print("Target Train : ", dec_train.shape)

# out:
# Source Train: (124960, 14)
# Target Train: (124960, 14)

## 만약 결과가 다르면 동일한 결과를 얻도록 해야함
## 학습데이터 개수가 124960 보다 크다면 step3다시 확인

# Step 5. 인공지능 만들기

In [ ]:
# 인공지능 만들기
# 모델의 embedding size와 hideen size를 조절하여 10epoch안에 val_loss값을 2.2 수준으로 줄일 수 잇는 모델 설계
# loss는 아래 제시된 loss함수 그대로 사용

# loss = tf.keras.losses.SparseCategoricalCrossentropy(
#     from_logits=True, reduction='none')

# generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

# 공백으로 나눠진 문자열에서 단어가 몇개있는지 헤아리기!